<a href="https://colab.research.google.com/github/Raj0408/Movie_Recommendation/blob/main/movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate

pd.set_option("display.precision", 1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256419 sha256=cd3eff9b7bf8b95e71e8adbd3d3927d43e869f1d06c378274149c417971546c5
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
top10_df = pd.read_csv("content_top10_df.csv")
bygenre_df = pd.read_csv("content_bygenre_df.csv")
top10_df

,movie id,num ratings,ave rating,title,genres
0,4993,198,4.1,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
1,5952,188,4.0,"Lord of the Rings: The Two Towers, The",Adventure|Fantasy
2,7153,185,4.1,"Lord of the Rings: The Return of the King, The",Action|Adventure|Drama|Fantasy
3,4306,170,3.9,Shrek,Adventure|Animation|Children|Comedy|Fantasy|Ro...
4,58559,149,4.2,"Dark Knight, The",Action|Crime|Drama
5,6539,149,3.8,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
6,79132,143,4.1,Inception,Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,6377,141,4.0,Finding Nemo,Adventure|Animation|Children|Comedy
8,4886,132,3.9,"Monsters, Inc.",Adventure|Animation|Children|Comedy|Fantasy
9,7361,131,4.2,Eternal Sunshine of the Spotless Mind,Drama|Romance|Sci-Fi


In [ ]:
bygenre_df

,genre,num movies,ave rating/genre,ratings per genre
0,Action,321,3.4,10377
1,Adventure,234,3.4,8785
2,Animation,76,3.6,2588
3,Children,69,3.4,2472
4,Comedy,326,3.4,8911
5,Crime,139,3.5,4671
6,Documentary,13,3.8,280
7,Drama,342,3.6,10201
8,Fantasy,124,3.4,4468
9,Horror,56,3.2,1345


In [ ]:
from collections import defaultdict
import csv
import numpy as np
from numpy import genfromtxt
import pickle5 as pickle
import tabulate

def load_data():
    ''' called to load preprepared data for the lab '''
    item_train = genfromtxt('content_item_train.csv', delimiter=',')
    user_train = genfromtxt('content_user_train.csv', delimiter=',')
    y_train    = genfromtxt('content_y_train.csv', delimiter=',')
    with open('content_item_train_header.txt', newline='') as f:    #csv reader handles quoted strings better
        item_features = list(csv.reader(f))[0]
    with open('content_user_train_header.txt', newline='') as f:
        user_features = list(csv.reader(f))[0]
    item_vecs = genfromtxt('content_item_vecs.csv', delimiter=',')

    movie_dict = defaultdict(dict)
    count = 0
#    with open('./data/movies.csv', newline='') as csvfile:
    with open('content_movie_list.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1  #skip header
                #print(line) print
            else:
                count += 1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]

    with open('content_user_to_genre.pickle', 'rb') as f:
        user_to_genre = pickle.load(f)

    return(item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre)

In [ ]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3
num_item_features = item_train.shape[1] - 1
uvs = 3
ivs = 3
u_s = 3
i_s = 1
print(f"Number of training vectors: {len(item_train)}")

Number of training vectors: 50884


In [ ]:
print(f"y_train[:5]: {y_train[:5]}")

y_train[:5]: [4.  3.5 4.  4.  4.5]


In [ ]:

item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

print(np.allclose(item_train_unscaled, scalerItem.inverse_transform(item_train)))
print(np.allclose(user_train_unscaled, scalerUser.inverse_transform(user_train)))

True
True


In [ ]:
item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (40707, 17)
movie/item test data shape: (10177, 17)


In [ ]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([

    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dense(num_outputs)




])

item_NN = tf.keras.models.Sequential([

    tf.keras.layers.Dense(256,activation = 'relu'),
    tf.keras.layers.Dense(128,activation = 'relu'),
    tf.keras.layers.Dense(num_outputs)



])


input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)
output = tf.keras.layers.Dot(axes=1)([vu, vm])

model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 14)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           40864       ['input_1[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 32)           41376       ['input_2[0][0]']                
                                                                                              

In [ ]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [ ]:
tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1273/1273 [==============================] - 7s 4ms/step - loss: 0.1241
Epoch 2/30
1273/1273 [==============================] - 4s 3ms/step - loss: 0.1144
Epoch 3/30
1273/1273 [==============================] - 4s 3ms/step - loss: 0.1101
Epoch 4/30
1273/1273 [==============================] - 6s 4ms/step - loss: 0.1062
Epoch 5/30
1273/1273 [==============================] - 4s 4ms/step - loss: 0.1031
Epoch 6/30
1273/1273 [==============================] - 5s 4ms/step - loss: 0.1001
Epoch 7/30
1273/1273 [==============================] - 5s 4ms/step - loss: 0.0976
Epoch 8/30
1273/1273 [==============================] - 4s 4ms/step - loss: 0.0954
Epoch 9/30
1273/1273 [==============================] - 6s 4ms/step - loss: 0.0935
Epoch 10/30
1273/1273 [==============================] - 4s 3ms/step - loss: 0.0919
Epoch 11/30
1273/1273 [==============================] - 4s 4ms/step - loss: 0.0901
Epoch 12/30
1273/1273 [==============================] - 6s 4ms/step - loss: 0.0883
E

In [16]:
model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], y_test)

319/319 [==============================] - 1s 3ms/step - loss: 0.0823


0.08231744170188904

In [52]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 10.0
new_childrens = 5.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 0.0
new_fantasy = 5.0
new_horror = 5.0
new_mystery = 5.0
new_romance = 0.0
new_scifi = 5.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [68]:
def gen_user_vecs(user_vec, num_items):

    user_vecs = np.tile(user_vec, (num_items, 1))
    return user_vecs

# predict on  everything, filter on print/use
def predict_uservec(user_vecs, item_vecs, model, u_s, i_s, scaler):

    y_p = model.predict([user_vecs[:, u_s:], item_vecs[:, i_s:]])
    y_pu = scaler.inverse_transform(y_p)

    if np.any(y_pu < 0):
        print("Error, expected all positive predictions")
    sorted_index = np.argsort(-y_pu, axis=0).reshape(-1).tolist()  #negate to get largest rating first
    sorted_ypu   = y_pu[sorted_index]
    sorted_items = item_vecs[sorted_index]
    sorted_user  = user_vecs[sorted_index]
    return(sorted_index, sorted_ypu, sorted_items, sorted_user)

def get_user_vecs(user_id, user_train, item_vecs, user_to_genre):

    if not user_id in user_to_genre:
        print("error: unknown user id")
        return None
    else:
        user_vec_found = False
        for i in range(len(user_train)):
            if user_train[i, 0] == user_id:
                user_vec = user_train[i]
                user_vec_found = True
                break
        if not user_vec_found:
            print("error in get_user_vecs, did not find uid in user_train")
        num_items = len(item_vecs)
        user_vecs = np.tile(user_vec, (num_items, 1))

        y = np.zeros(num_items)
        for i in range(num_items):  # walk through movies in item_vecs and get the movies, see if user has rated them
            movie_id = item_vecs[i, 0]
            if movie_id in user_to_genre[user_id]['movies']:
                rating = user_to_genre[user_id]['movies'][movie_id]
            else:
                rating = 0
            y[i] = rating
    return(user_vecs, y)

def get_item_genres(item_gvec, genre_features):

    offsets = np.nonzero(item_gvec)[0]
    genres = [genre_features[i] for i in offsets]
    return genres


def print_existing_user(y_p, y, user, items, ivs, uvs, movie_dict, maxcount=10):

    count = 0
    disp = [["y_p", "y", "user", "user genre ave", "movie rating ave", "movie id", "title", "genres"]]
    count = 0
    for i in range(0, y.shape[0]):
        if y[i, 0] != 0:  # zero means not rated
            if count == maxcount:
                break
            count += 1
            movie_id = items[i, 0].astype(int)

            offsets = np.nonzero(items[i, ivs:] == 1)[0]
            genre_ratings = user[i, uvs + offsets]
            disp.append([y_p[i, 0], y[i, 0],
                         user[i, 0].astype(int),      # userid
                         np.array2string(genre_ratings,
                                         formatter={'float_kind':lambda x: "%.1f" % x},
                                         separator=',', suppress_small=True),
                         items[i, 2].astype(float),    # movie average rating
                         movie_id,
                         movie_dict[movie_id]['title'],
                         movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow", floatfmt=[".1f", ".1f", ".0f", ".2f", ".1f"])
    return table

def print_pred_movies(y_p, item, movie_dict, maxcount=10):

    count = 0
    disp = [["y_p", "movie id", "rating ave", "title", "genres"]]

    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int)
        disp.append([np.around(y_p[i, 0], 1), item[i, 0].astype(int), np.around(item[i, 2].astype(float), 1),
                     movie_dict[movie_id]['title'], movie_dict[movie_id]['genres']])

    table = tabulate.tabulate(disp, tablefmt='txt', headers="firstrow")
    return table


In [62]:
from IPython.display import IFrame

In [64]:
IFrame(print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10), width=700, height=600)

y_p,movie id,rating ave,title,genres
4.8,108932,3.9,The Lego Movie (2014),Action|Adventure|Animation|Children|Comedy|Fantasy
4.7,5072,4,Metropolis (2001),Animation|Sci-Fi
4.6,5444,3.8,Lilo & Stitch (2002),Adventure|Animation|Children|Sci-Fi
4.6,5618,4.2,Spirited Away (Sen to Chihiro no kamikakushi) (2001),Adventure|Animation|Fantasy
4.6,6283,3.9,Cowboy Bebop: The Movie (Cowboy Bebop: Tengoku no Tobira) (2001),Action|Animation|Sci-Fi|Thriller
4.5,50872,3.9,Ratatouille (2007),Animation|Children|Drama
4.5,4886,3.9,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy
4.5,115617,3.9,Big Hero 6 (2014),Action|Animation|Comedy
4.5,60069,4.1,WALL·E (2008),Adventure|Animation|Children|Romance|Sci-Fi
4.5,27773,4.1,Old Boy (2003),Mystery|Thriller


In [67]:
import IPython
IPython.display.HTML(filename=print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10))

FileNotFoundError: ignored

In [69]:

user_vecs = gen_user_vecs(user_vec,len(item_vecs))

suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)

y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction
y_pu = scalerTarget.inverse_transform(y_p)

sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]

sorted_items
print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

27/27 [==============================] - 0s 2ms/step


'  y_p    movie id    rating ave  title                                                             genres\n-----  ----------  ------------  ----------------------------------------------------------------  --------------------------------------------------\n  4.8      108932           3.9  The Lego Movie (2014)                                             Action|Adventure|Animation|Children|Comedy|Fantasy\n  4.7        5072           4    Metropolis (2001)                                                 Animation|Sci-Fi\n  4.6        5444           3.8  Lilo & Stitch (2002)                                              Adventure|Animation|Children|Sci-Fi\n  4.6        5618           4.2  Spirited Away (Sen to Chihiro no kamikakushi) (2001)              Adventure|Animation|Fantasy\n  4.6        6283           3.9  Cowboy Bebop: The Movie (Cowboy Bebop: Tengoku no Tobira) (2001)  Action|Animation|Sci-Fi|Thriller\n  4.5       50872           3.9  Ratatouille (2007)                         

### Preditcing for existing user

In [54]:
uid = 2
user_vecs, y_vecs = get_user_vecs(uid, user_train_unscaled, item_vecs, user_to_genre)

suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerItem.transform(item_vecs)


y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

y_pu = scalerTarget.inverse_transform(y_p)


sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()
sorted_ypu   = y_pu[sorted_index]
sorted_items = item_vecs[sorted_index]
sorted_user  = user_vecs[sorted_index]
sorted_y     = y_vecs[sorted_index]




27/27 [==============================] - 0s 2ms/step


### Recommendation of movie based on user profile

In [44]:
def sq_dist(a,b):
    d= np.sum((a-b)**2)
    return d

In [56]:
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)
model_m.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential_1 (Sequential)   (None, 32)                41376     
                                                                 
 tf.math.l2_normalize_5 (TFO  (None, 32)               0         
 pLambda)                                                        
                                                                 
Total params: 41,376
Trainable params: 41,376
Non-trainable params: 0
_________________________________________________________________


In [57]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

27/27 [==============================] - 0s 1ms/step
size of all predicted movie feature vectors: (847, 32)


In [58]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])

m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal

disp = [["movie1", "genres", "movie2", "genres"]]
for i in range(count):
    min_idx = np.argmin(m_dist[i])
    movie1_id = int(item_vecs[i,0])
    movie2_id = int(item_vecs[min_idx,0])
    disp.append( [movie_dict[movie1_id]['title'], movie_dict[movie1_id]['genres'],
                  movie_dict[movie2_id]['title'], movie_dict[movie1_id]['genres']]
               )

table = tabulate.tabulate(disp, headers="firstrow")
table

"movie1                                    genres                                               movie2                                                       genres\n----------------------------------------  ---------------------------------------------------  -----------------------------------------------------------  ---------------------------------------------------\nSave the Last Dance (2001)                Drama|Romance                                        Mona Lisa Smile (2003)                                       Drama|Romance\nWedding Planner, The (2001)               Comedy|Romance                                       Mr. Deeds (2002)                                             Comedy|Romance\nHannibal (2001)                           Horror|Thriller                                      Final Destination 2 (2003)                                   Horror|Thriller\nSaving Silverman (Evil Woman) (2001)      Comedy|Romance                                       Sweetest Thing,

In [49]:
disp

[['movie1', 'genres', 'movie2', 'genres'],
 ['Save the Last Dance (2001)',
  'Drama|Romance',
  'Mona Lisa Smile (2003)',
  'Drama|Romance'],
 ['Wedding Planner, The (2001)',
  'Comedy|Romance',
  'Mr. Deeds (2002)',
  'Comedy|Romance'],
 ['Hannibal (2001)',
  'Horror|Thriller',
  'Final Destination 2 (2003)',
  'Horror|Thriller'],
 ['Saving Silverman (Evil Woman) (2001)',
  'Comedy|Romance',
  'Sweetest Thing, The (2002)',
  'Comedy|Romance'],
 ['Down to Earth (2001)',
  'Comedy|Fantasy|Romance',
  'Bewitched (2005)',
  'Comedy|Fantasy|Romance'],
 ['Mexican, The (2001)',
  'Action|Comedy',
  'Rush Hour 2 (2001)',
  'Action|Comedy'],
 ['15 Minutes (2001)', 'Thriller', 'Panic Room (2002)', 'Thriller'],
 ['Enemy at the Gates (2001)', 'Drama', 'Seabiscuit (2003)', 'Drama'],
 ['Heartbreakers (2001)',
  'Comedy|Crime|Romance',
  'Fun with Dick and Jane (2005)',
  'Comedy|Crime|Romance'],
 ['Spy Kids (2001)',
  'Action|Adventure|Children|Comedy',
  'Scooby-Doo (2002)',
  'Action|Adventure|Ch